<a href="https://colab.research.google.com/github/momo54/minfo2022/blob/main/MasterInfo_UE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# grrr

Bon...
L'extraction excel de pdf ne peut être miraculeux..

Quand un tableau est à cheval sur plusieurs page, le tableau est mal reconstitué...

Il faut avoir les tableaux d'origine !!

Donc un read-Excel ne pourra donner de bons résultats...

In [1]:
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 57 kB/s 
     |████████████████████████████████| 199 kB 68.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=27c7987c8f602d0fe9582e971299ca93337eda6b70377ce08e3f7473d5e3abad
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
import pandas as pd

##
## Autre Approche
## Même si 1 sheet != 1 UE
## l'odre est préservé
## regexp extraction en suivant l'ordre...

my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

df=pd.read_excel(my_csv, index_col=None, header=None, sheet_name="Table 5")
#Title and code of UE not at the right Place :(
# need swapping...
#print(df)
#print(df.dtypes)
mask=df.apply(lambda x: x.str.contains('X1.*|Objectifs|Contenu',regex=True)).any(axis=1)
print(df[mask])



                                        0  \
1                                 X1II010   
14  Objectifs (résultats d'apprentissage)   
15                                Contenu   
19                                X1II030   
32  Objectifs (résultats d'apprentissage)   
33                                Contenu   

                                                    1  
1                                             Graphes  
14  1. reconnaître et modéliser un problème réel a...  
15  •\nEléments de théorie des graphes\n•\nArbres ...  
19                          Complexité et algorithmes  
32  • Savoir manipuler les notations standards des...  
33                                                NaN  


In [20]:
#
# DS M2 et Vico M2 n'ont pas les code UE pour les UE...
#

parcours=[
    ('ALMA','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'),
    ('ALMA','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx'),
    ('ATAL','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx'),
    ('ATAL','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023.xlsx'),
    ('ORO','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx'),
    ('ORO','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx'),
    ('DS','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Data_Science_(DS)_2022-2023.xlsx'),
    ('DS','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Data_Science_(DS)_2022-2023-withcode.xlsx'),
    ('VICO','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Visual_Computing_(VICO)_2022-2023.xlsx'),
    ('VICO','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Visual_Computing_(VICO)-withcode.xlsx'),
]

def extract_ue(parcours,an,path):
  print(f"extracting ues for {parcours}, annee:{an}, path:{path}")
  dfs=[]
#  my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

  for sheet_name, df in pd.read_excel(path, index_col=None, header=None, sheet_name=None).items():
      sheetid=int(sheet_name.split(" ")[1])
      if sheetid>4:
        print(sheet_name)
        dfs.append(df)
  allUE=pd.concat(dfs).reset_index(drop=True)

  mask=allUE.apply(lambda x: x.str.contains('^X[1-4].*|^ZZ|^YG.*|Object|Contenu',regex=True)).any(axis=1)
  UES=allUE[mask]

  #debug
  #print(UES.columns.values)
  #print(UES)
  #i don't know why i need 3 colmns sometimes...
  # read_excel pb ???
  if len(UES.columns.values)==2:
    UES.set_axis(['X','Y'],axis=1,inplace=True)
  else:
    UES.set_axis(['X','Y','Z'],axis=1,inplace=True)
    
  #debug
  #print(UES.columns.values)
  #print(UES)
  #for index, row in UES.iterrows():
  #  print(f"extracting:{row['X']}")

  state=0
  key=""
  title=""
  objective=""
  content=""
  lines=[]

  import re
  reg=re.compile("^X[1-4].*|^ZZ|^YG.*")

  for index, row in UES.iterrows():
    #print(f"extracting:{row['X']}")
    #print(row['X'],row['Y']
    if state==0:
      if reg.match(str(row['X'])) is not None:
        key=row['X']
        title=row['Y']
        state=1
      else:
        print(f"error: should read an UE key:{row['X']}")
        state=0
    elif state==1:
      if str(row['X']).startswith("Object"):
        objective=row['Y']
        state=2
      else:
        print(f"error: should read an Objective:{row['X']}")
        state=0
    elif state==2:
      if str(row['X']).startswith("Contenu"):
        content=row['Y']
        lines.append({'key' : key, 'title':title,'objective':objective,'content':content,'parc':parcours,'an':an,'from':path})
        state=0
      else:
        print(f"error: should read an Content:{row['X']}")
        state=0
        
  out=pd.DataFrame(lines)
  out.to_csv(f"{parcours}_{an}_UEs.csv",sep='|',index=False,header=True)

for p,a,path in parcours:
  extract_ue(p,a,path)

extracting ues for ALMA, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
Table 12
Table 13
Table 14
Table 15
Table 16
Table 17
Table 18
Table 19
Table 20
Table 21
Table 22
Table 23
Table 24
extracting ues for ALMA, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
extracting ues for ATAL, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
Table 12
Table 13
Table 14
Table 15
Table 16
Table 17
Table 18
Table 19
Table 20
Table 21
Table 22
Table 23
Table 24
extracting ues for ATAL, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(A

In [21]:
# OK CSV shoud be generated now

from pyspark.sql.types import *
schema = StructType([
    StructField("key",StringType(),True), 
    StructField("title", StringType(), True),
    StructField("objective", StringType(), True),
    StructField("content", StringType(), True),
    StructField("parc", StringType(), True),
    StructField("an", StringType(), True),
    StructField("path", StringType(), True),
    ])

file_list=[]
for pa,an,path in parcours:
  file_list.append(f"/content/{pa}_{an}_UEs.csv")

print(','.join(file_list))

data = spark.read \
    .format("csv")   \
    .option("delimiter","|") \
    .option("multiLine", True)\
    .option("header",True) \
    .schema(schema) \
    .load(file_list)
print(data.count())
data.show(209)

data.createOrReplaceTempView(f"UES")

spark.sql("select key,title,objective, collect_set(parc), collect_set(an) from UES  group by key,title,objective,content").show(200)




/content/ALMA_M1_UEs.csv,/content/ALMA_M2_UEs.csv,/content/ATAL_M1_UEs.csv,/content/ATAL_M2_UEs.csv,/content/ORO_M1_UEs.csv,/content/ORO_M2_UEs.csv,/content/DS_M1_UEs.csv,/content/DS_M2_UEs.csv,/content/VICO_M1_UEs.csv,/content/VICO_M2_UEs.csv
205
+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+
|                 key|               title|           objective|             content|                parc|  an|                path|
+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+
|             X1II010|             Graphes|1. reconnaître et...|•\nEléments de th...|                ATAL|  M1|/content/drive/My...|
|             X1II030|Complexité et alg...|• Savoir manipule...|                null|                ATAL|  M1|/content/drive/My...|
|             X1II040|Anglais scientifique|Au terme de cette...|Anglais de spécia...|  

In [ ]:
##
## MArche pas... 1 sheet != UE

from re import VERBOSE
import pandas as pd

!rm ALMA_M1*

result=pd.DataFrame()

my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'
for sheet_name, df in pd.read_excel(my_csv, index_col=0, header=None, sheet_name=None).items():
    sheetid=int(sheet_name.split(" ")[1])
    if sheetid>4:
      print(sheet_name)
      df=df.transpose()
      df.iat[0,0]=df.columns.values[1]
      df.columns.values[1]="Titre"
      df.to_csv(f'ALMA_M1_{sheetid}.csv', index=False, encoding='utf-8')

#sheet.info(verbose=True)
# Not the same fields in all sheets :(
# result=pd.concat([result,sheet]).reset_index(drop=True)
result